In [5]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import functions

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [6]:
cap = cv2.VideoCapture(0)


counter = 0
stage = None
minAngle = 180
maxAngle = 0

#Set up different Joints
JOINTS_TO_TRACK = {
    "left_elbow": [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST],
    "right_elbow": [mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST],
    "left_knee": [mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE],
    "right_knee": [mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE],
    # Add more as needed
}


#initializes a map to store the x,y,z coordinate of each body part at each frame
landmark_positions = {}
for position in mp_pose.PoseLandmark:
    landmark_positions[position.value] = [[],[],[]]


#Stores the angles calculated at each frame in a different map
angle_history = {joint: [] for joint in JOINTS_TO_TRACK}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        #Recolor the frame to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        #Recolor the frame back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)


        #Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            for position in mp_pose.PoseLandmark:
                landmark = landmarks[position.value]
                landmark_positions[position.value][0].append(round(landmark.x,2))
                landmark_positions[position.value][1].append(round(landmark.y,2))
                landmark_positions[position.value][2].append(round(landmark.z,2))

            #LEFT ARM
            for joint_name, landmark_indices in JOINTS_TO_TRACK.items():
                try:
                    a = landmarks[landmark_indices[0].value]
                    b = landmarks[landmark_indices[1].value]
                    c = landmarks[landmark_indices[2].value]

                    # Use only x, y for 2D analysis
                    angle = functions.calculate_angle(
                        [a.x, a.y, a.z],
                        [b.x, b.y, b.z],
                        [c.x, c.y, c.z]
                    )
                except Exception:
                    angle = np.nan

                angle_history[joint_name].append(int(angle))

                if not np.isnan(angle):
                    b_coords = np.multiply([b.x, b.y], [image.shape[1], image.shape[0]]).astype(int)
                    cv2.putText(
                        image,
                        f"{joint_name}: {int(angle)}",
                        tuple(b_coords),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (255,255,255),
                        2,
                        cv2.LINE_AA
                    )



            #Visualize angle
            cv2.putText(image, str(angle),
                            tuple(np.multiply(elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                                  )
            #Curl Counter Logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == "down":
                stage = "up"
                counter += 1
                print(counter)
            prevAngle = angle

            #########################################################################################################################
            '''hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]


            angle = calculate_angle(hip, shoulder, elbow)

            #Visualize angle
            cv2.putText(image, str(angle),
                            tuple(np.multiply(elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                                  )'''

        except:
            pass

        #Status Box and Rep Counter
        cv2.rectangle(image, (0,0), (255,73), (245,117,16), -1)
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 1, cv2.LINE_AA)
        '''cv2.putText(image, str(round(minAngle,2)), (10,100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(maxAngle,2)), (10,150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)'''


        #Render detections  (image, landmarks e.g. body part positions, landmark connections e.g. nose to eye)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


    points = {f"landmark_{k}": np.array(v) for k, v in landmark_positions.items()}
    angles = {k:np.array(v) for k, v in angle_history.items()}
    np.savez("points.npz", **points)
    np.savez("angles.npz", **angles)
    cap.release()
    cv2.destroyAllWindows()

In [7]:
import numpy as np

data = np.load("data.npz")

#format: x,y,z: coordinates for each. divide by 60 to find how long recording is
'''for key in data.files:
    print(key, data[key].shape)'''

#maps each body position to its index defined my MediaPipe
index_to_name = {str(i): landmark.name for i, landmark in enumerate(mp_pose.PoseLandmark)}

# Print the landmark name and a few values
for key in data.files:

    idx = key.split('_')[-1]  # Extracts index from landmark, key is always last digit or digits
    name = index_to_name.get(idx, f"Unknown({idx})")
    #displays the Name of the body part, the key associated with it (e.g. landmark 5), and the last 5
    #values of x,y,z for that landmark
    print(f"{name} ({key}):" + "\n" + f"{data[key][:, :5]}")

angles = np.load('angles.npz')
for key in angles:
    print(key, angles[key][:5])

#Very easy to see when a rep is at the bottom versus at the top.
print(angles['left_elbow'])

NOSE (landmark_0):
[[ 0.61  0.61  0.61  0.6   0.6 ]
 [ 0.16  0.13  0.12  0.12  0.12]
 [-0.29 -0.02 -0.31 -0.2   0.11]]
LEFT_EYE_INNER (landmark_1):
[[ 0.61  0.61  0.61  0.61  0.61]
 [ 0.09  0.08  0.07  0.07  0.07]
 [-0.29  0.01 -0.26 -0.17  0.16]]
LEFT_EYE (landmark_2):
[[ 0.62  0.62  0.62  0.62  0.61]
 [ 0.1   0.08  0.07  0.07  0.08]
 [-0.29  0.01 -0.26 -0.18  0.16]]
LEFT_EYE_OUTER (landmark_3):
[[ 0.62  0.62  0.62  0.62  0.62]
 [ 0.1   0.09  0.08  0.08  0.08]
 [-0.29  0.01 -0.26 -0.18  0.16]]
RIGHT_EYE_INNER (landmark_4):
[[ 0.59  0.59  0.59  0.59  0.58]
 [ 0.11  0.07  0.05  0.05  0.05]
 [-0.33 -0.03 -0.32 -0.22  0.12]]
RIGHT_EYE (landmark_5):
[[ 0.59  0.58  0.58  0.58  0.57]
 [ 0.11  0.06  0.05  0.05  0.06]
 [-0.33 -0.03 -0.32 -0.22  0.12]]
RIGHT_EYE_OUTER (landmark_6):
[[ 0.58  0.57  0.56  0.56  0.56]
 [ 0.11  0.09  0.07  0.07  0.08]
 [-0.33 -0.03 -0.32 -0.22  0.12]]
LEFT_EAR (landmark_7):
[[ 0.59  0.59  0.59  0.59  0.59]
 [ 0.12  0.09  0.08  0.08  0.08]
 [-0.18  0.17  0.16  0.01  